In [1]:
import os
from dotenv import load_dotenv
from aocd import get_data

In [2]:
load_dotenv()
secret_key = os.getenv("SECRET_KEY")
os.environ['AOC_SESSION'] = secret_key

data = get_data(year=2023, day=9)

## Part 1
The OASIS produces a report of many values and how they are changing over time (your puzzle input). Each line in the report contains the history of a single value. For example:
```
0 3 6 9 12 15
1 3 6 10 15 21
10 13 16 21 30 45
```
To best protect the oasis, your environmental report should include a prediction of the next value in each history. To do this, start by making a new sequence from the difference at each step of your history. If that sequence is not all zeroes, repeat this process, using the sequence you just generated as the input sequence. Once all of the values in your latest sequence are zeroes, you can extrapolate what the next value of the original history should be.

In the above dataset, the first history is 0 3 6 9 12 15. Because the values increase by 3 each step, the first sequence of differences that you generate will be 3 3 3 3 3. Note that this sequence has one fewer value than the input sequence because at each step it considers two numbers from the input. Since these values aren't all zero, repeat the process: the values differ by 0 at each step, so the next sequence is 0 0 0 0. This means you have enough information to extrapolate the history! Visually, these sequences can be arranged like this:
```
0   3   6   9  12  15
  3   3   3   3   3
    0   0   0   0
```
To extrapolate, start by adding a new zero to the end of your list of zeroes; because the zeroes represent differences between the two values above them, this also means there is now a placeholder in every sequence above it:
```
0   3   6   9  12  15   B
  3   3   3   3   3   A
    0   0   0   0   0
```
You can then start filling in placeholders from the bottom up. A needs to be the result of increasing 3 (the value to its left) by 0 (the value below it); this means A must be 3:
```
0   3   6   9  12  15   B
  3   3   3   3   3   3
    0   0   0   0   0
```
Finally, you can fill in B, which needs to be the result of increasing 15 (the value to its left) by 3 (the value below it), or 18:
```
0   3   6   9  12  15  18
  3   3   3   3   3   3
    0   0   0   0   0
```
So, the next value of the first history is 18.

Finding all-zero differences for the second history requires an additional sequence:
```
1   3   6  10  15  21
  2   3   4   5   6
    1   1   1   1
      0   0   0
```
Then, following the same process as before, work out the next value in each sequence from the bottom up:
```
1   3   6  10  15  21  28
  2   3   4   5   6   7
    1   1   1   1   1
      0   0   0   0
```
So, the next value of the second history is 28.

The third history requires even more sequences, but its next value can be found the same way:
```
10  13  16  21  30  45  68
   3   3   5   9  15  23
     0   2   4   6   8
       2   2   2   2
         0   0   0
```
So, the next value of the third history is 68.

If you find the next value for each history in this example and add them together, you get 114.

Analyze your OASIS report and extrapolate the next value for each history. What is the sum of these extrapolated values?

In [3]:
def get_diff(series):
    diff = [int(series[i]) - int(series[i-1]) for i in range(1, len(series))]
    
    return diff

def generate_levels(series):
    level = 0
    curr_series = series
    series_levels = {level: curr_series}

    while not all(x == 0 for x in curr_series):
        level += 1
        diff = get_diff(curr_series)
        series_levels[level] = diff
        curr_series = diff

    return series_levels

def backpropagate(series_levels):
    new_levels = series_levels
    if len(new_levels.keys()) == 1:
        new_levels[0].append(0)
        return new_levels

    # Starting at the 2nd-to-last level and working up
    for i in list(new_levels.keys())[:-1][::-1]:
        # Grab last val from previous level
        new_val = new_levels[i+1][-1]
        # Append to series
        new_levels[i].append(new_levels[i][-1] + new_val)

    return new_levels

def parse_data(data):
    series_dict = {}
    for i,line in enumerate(data.splitlines()):
        series = [int(x) for x in line.split()]
        series_levels = generate_levels(series)
        new_levels = backpropagate(series_levels)
        series_dict[i] = new_levels
    
    return series_dict

def solve_part1(data):
    series_dict = parse_data(data)
    return sum([series_dict[i][0][-1] for i in series_dict])

In [4]:
solve_part1(data)

2105961943

## Part 2
Of course, it would be nice to have even more history included in your report. Surely it's safe to just extrapolate backwards as well, right?

For each history, repeat the process of finding differences until the sequence of differences is entirely zero. Then, rather than adding a zero to the end and filling in the next values of each previous sequence, you should instead add a zero to the beginning of your sequence of zeroes, then fill in new first values for each previous sequence.

In particular, here is what the third example history looks like when extrapolating back in time:
```
5  10  13  16  21  30  45
  5   3   3   5   9  15
   -2   0   2   4   6
      2   2   2   2
        0   0   0
```
Adding the new values on the left side of each sequence from bottom to top eventually reveals the new left-most history value: 5.

Doing this for the remaining example data above results in previous values of -3 for the first history and 0 for the second history. Adding all three new values together produces 2.

Analyze your OASIS report again, this time extrapolating the previous value for each history. What is the sum of these extrapolated values?

In [5]:
def backpropagate_v2(series_levels):
    new_levels = series_levels
    if len(new_levels.keys()) == 1:
        new_levels[0].append(0)
        return new_levels

    # Starting at the 2nd-to-last level and working up
    for i in list(new_levels.keys())[:-1][::-1]:
        # Grab first val from previous level
        new_val = new_levels[i+1][0]
        # Append to FRONT of series
        new_levels[i].insert(0, new_levels[i][0] - new_val)

    return new_levels

def parse_data_v2(data):
    series_dict = {}
    for i,line in enumerate(data.splitlines()):
        series = [int(x) for x in line.split()]
        series_levels = generate_levels(series)
        new_levels = backpropagate_v2(series_levels)
        series_dict[i] = new_levels
    
    return series_dict

def solve_part2(data):
    series_dict = parse_data_v2(data)
    return sum([series_dict[i][0][0] for i in series_dict])

In [6]:
solve_part2(data)

1019